# INFORMATION RETRIEVAL PROJECT
# 6. WORDS DEBIASING 

---
## Gender stereotypes in parliamentary speeches

In word embedding models, each word is assigned to a high-dimensional vector such that the geometry of the vectors captures semantic relations between the words – e.g. vectors being closer together has been shown to correspond to more similar words. Recent works in machine learning demonstrate that word embeddings also capture common stereotypes, as these stereotypes are likely to be present, even if subtly, in the large corpora of training texts. These stereotypes are automatically learned by the embedding algorithm and could be problematic in many context if the embedding is then used for sensitive applications such as search rankings, product recommendations, or translations. An important direction of research is on developing algorithms to debias the word embeddings.

This project aims to use the word embeddings to study historical trends – specifically trends in the gender and ethnic stereotypes in the Italian parliamentary speeches from 1948 to 2020.

In [1]:
import numpy as np
import pandas as pd
import gensim
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import os
from collections import defaultdict, OrderedDict
from tqdm.auto import tqdm

from INFORET_project import load_embed_model
# import matplotlib.pylab as plt
pd.set_option("display.max_rows", 100, "display.max_columns", 100)

In [2]:
from INFORET_project import YEARS

In [3]:
YEARS

['1948_1968', '1968_1985', '1985_2000', '2000_2020']

In [4]:
model = load_embed_model(YEARS[0])

In [7]:
model = load_embed_model(YEARS[1])

In [8]:
model = load_embed_model(YEARS[2])

In [16]:
model = load_embed_model(YEARS[3])

---

Use bolukbasi hard debiasing algorithm to remove gender stereotypes. Try different ways to compute the gender direction: single, sum and pca.

In [5]:
from INFORET_project import Debias
from INFORET_project.data import gendered_neutral_words

In [6]:
d = Debias(model.wv, definitional_pairs=['uomo','donna'],
                              identify_direction='single')
debiased_model_single = d.hard_debias(equality_sets=gendered_neutral_words['equality_sets'])

In [8]:
debiased_model_single

In [9]:
d = Debias(model.wv, definitional_pairs=['uomo','donna'],
                              identify_direction='sum')
debiased_model_sum = d.hard_debias(equality_sets=gendered_neutral_words['equality_sets'])

In [10]:
debiased_model_sum

In [11]:
d = Debias(model.wv, definitional_pairs=['uomo','donna'], identify_direction='pca')
debiased_model_pca = d.hard_debias(equality_sets=gendered_neutral_words['equality_sets'])

In [12]:
debiased_model_pca

---

Use EAB to see whether the hard debiasing reduces the bias within the language. 

Apply EAB on word2vec models before and after the debisaing. Print only the top biased word to see whether the bias is reduced

In [13]:
from INFORET_project.data import gendered_neutral_words
from INFORET_project import EAB
from INFORET_project import WORDS_GROUP

In [14]:
# print the top biased word of each group of words for EAB with averaged gendered vectors, before and 
#after debiasing

for year in tqdm(YEARS,
                 desc='Passing years'):
    print(f'\nYEAR: {year}')
    model = load_embed_model(year)
    
    for word in tqdm(WORDS_GROUP,
                     desc='Passing group of words'):
        print(f'\n\nGROUP OF WORDS: {word}\n')

        print('\nMODEL: Not debiased model')
        score = EAB(model.wv,
                    use_avg_gender=True,
                    type_most_similar = 'cosmul')
        # return only the top biased word for each group of words
        top_bias = score.get_top_bias(pred_positive_word=word, verbose=False)[0]
        # print the word and the total bias (difference of the cosine similarities btw words
        #returned by the analogy and genders)
        print( (top_bias[0], abs(top_bias[1][0] - top_bias[1][1])) )
        
        for deb_type in ['single', 'sum', 'pca']:
            d = Debias(model.wv, definitional_pairs=['uomo','donna'],
                              identify_direction=deb_type)
            # return debiased word2vec model
            debiased_model = d.hard_debias(equality_sets=gendered_neutral_words['equality_sets'])
            
            score = EAB(debiased_model,
                        use_avg_gender=True,
                        type_most_similar = 'cosmul')
            print(f'\nMODEL: Debiased model {deb_type}')
            top_bias = score.get_top_bias(pred_positive_word=word, verbose=False)[0]
            print( (top_bias[0], abs(top_bias[1][0] - top_bias[1][1])) )



YEAR: 1948_1968




GROUP OF WORDS: adj_appearence


MODEL: Not debiased model
('grasso', 0.06127673238515857)

MODEL: Debiased model single
('splendido', 0.32155335918068884)

MODEL: Debiased model sum
('splendido', 0.04324311316013335)

MODEL: Debiased model pca
('brutto', 0.3194217950105668)


GROUP OF WORDS: family


MODEL: Not debiased model
('bambino', 0.04381868690252305)

MODEL: Debiased model single
('figlio', 0.2617485515773296)

MODEL: Debiased model sum
('matrimonio', 0.03625070452690127)

MODEL: Debiased model pca
('bambino', 0.294831258803606)


GROUP OF WORDS: career


MODEL: Not debiased model
('capo', 0.0394399270415306)

MODEL: Debiased model single
('capo', 0.3804219402372837)

MODEL: Debiased model sum
('capo', 0.03289711177349092)

MODEL: Debiased model pca
('capo', 0.3760174781084061)


GROUP OF WORDS: rage


MODEL: Not debiased model
('aggressivo', 0.0696475218981504)

MODEL: Debiased model single
('cattivo', 0.3830927975475788)

MODEL: Debiased model sum
('crudele', 0.04096414446



GROUP OF WORDS: adj_appearence


MODEL: Not debiased model
('sensuale', 0.03292045444250108)

MODEL: Debiased model single
('bello', 0.37385801970958715)

MODEL: Debiased model sum
('bello', 0.04820239841938018)

MODEL: Debiased model pca
('brutto', 0.015571779012680087)


GROUP OF WORDS: family


MODEL: Not debiased model
('bambino', 0.05103850066661836)

MODEL: Debiased model single
('figlio', 0.3037390366196633)

MODEL: Debiased model sum
('bambino', 0.022042566537857022)

MODEL: Debiased model pca
('accudire', 0.0025791302323340815)


GROUP OF WORDS: career


MODEL: Not debiased model
('presidente', 0.04962961599230764)

MODEL: Debiased model single
('professionale', 0.37142196595668797)

MODEL: Debiased model sum
('onorevole', 0.018504664301872253)

MODEL: Debiased model pca
('onorevole', 0.014784026145935103)


GROUP OF WORDS: rage


MODEL: Not debiased model
('intollerante', 0.0602129846811294)

MODEL: Debiased model single
('crudele', 0.3755185320973397)

MODEL: Debiased mode



GROUP OF WORDS: adj_appearence


MODEL: Not debiased model
('frivolo', 0.06346932649612425)

MODEL: Debiased model single
('grasso', 0.39070465769618745)

MODEL: Debiased model sum
('bello', 0.021702119708061185)

MODEL: Debiased model pca
('grasso', 0.3305662658065558)


GROUP OF WORDS: family


MODEL: Not debiased model
('bambino', 0.025714072585105885)

MODEL: Debiased model single
('famiglia', 0.3732928760349751)

MODEL: Debiased model sum
('genitore', 0.005285957455635093)

MODEL: Debiased model pca
('famiglia', 0.3127041898667812)


GROUP OF WORDS: career


MODEL: Not debiased model
('capo', 0.012652518227696419)

MODEL: Debiased model single
('onorevole', 0.3628251388669014)

MODEL: Debiased model sum
('carriera', 0.02822107672691343)

MODEL: Debiased model pca
('professionale', 0.3292255993932486)


GROUP OF WORDS: rage


MODEL: Not debiased model
('cattivo', 0.04554537683725357)

MODEL: Debiased model single
('brutale', 0.40178558081388477)

MODEL: Debiased model sum
('crude



GROUP OF WORDS: adj_appearence


MODEL: Not debiased model
('brutto', 0.03416837677359583)

MODEL: Debiased model single
('bello', 0.3236428740434349)

MODEL: Debiased model sum
('splendido', 0.07683547586202621)

MODEL: Debiased model pca
('bello', 0.6495590589009226)


GROUP OF WORDS: family


MODEL: Not debiased model
('bambino', 0.03165259659290315)

MODEL: Debiased model single
('famiglia', 0.37531661717221143)

MODEL: Debiased model sum
('matrimonio', 0.056497968733310755)

MODEL: Debiased model pca
('genitore', 0.6651592994108796)


GROUP OF WORDS: career


MODEL: Not debiased model
('capo', 0.0712145067751408)

MODEL: Debiased model single
('onorevole', 0.37300224630162115)

MODEL: Debiased model sum
('presidente', 0.02160630375146866)

MODEL: Debiased model pca
('professionale', 0.715283754747361)


GROUP OF WORDS: rage


MODEL: Not debiased model
('intollerante', 0.058555851876735676)

MODEL: Debiased model single
('brutale', 0.42053920058533545)

MODEL: Debiased model sum


In [15]:
# print the top biased word of each group of words for EAB without averaged gendered vectors, before and 
#after debiasing


for year in tqdm(YEARS,
                 desc='Passing years'):
    print(f'\nYEAR: {year}')
    model = load_embed_model(year)
    
    for word in tqdm(WORDS_GROUP,
                     desc='Passing group of words'):
        print(f'\n\nGROUP OF WORDS: {word}\n')

        print('\nMODEL: Not debiased model')
        score = EAB(model.wv,
                    gender_female='donna',
                    gender_male='uomo',
                    type_most_similar = 'cosmul')
        top_bias = score.get_top_bias(pred_positive_word=word, verbose=False)[0]
        print( (top_bias[0], abs(top_bias[1][0] - top_bias[1][1])) )
        
        for deb_type in ['single', 'sum', 'pca']:
            d = Debias(model.wv, definitional_pairs=['uomo','donna'],
                              identify_direction=deb_type)
            debiased_model = d.hard_debias(equality_sets=gendered_neutral_words['equality_sets'])
            
            score = EAB(debiased_model,
                        gender_female='donna',
                        gender_male='uomo',
                        type_most_similar = 'cosmul')
            print(f'\nMODEL: Debiased model {deb_type}')
            top_bias = score.get_top_bias(pred_positive_word=word, verbose=False)[0]
            print( (top_bias[0], abs(top_bias[1][0] - top_bias[1][1])) )



YEAR: 1948_1968




GROUP OF WORDS: adj_appearence


MODEL: Not debiased model
('brutto', 0.02924209404736755)

MODEL: Debiased model single
('splendido', 0.008128891512751646)

MODEL: Debiased model sum
('splendido', 0.011958493106067225)

MODEL: Debiased model pca
('rozzo', 0.03843159824609754)


GROUP OF WORDS: family


MODEL: Not debiased model
('accudire', 0.10531398877501488)

MODEL: Debiased model single
('accudire', 0.0032697383314370665)

MODEL: Debiased model sum
('accudire', 0.016254047863185406)

MODEL: Debiased model pca
('accudire', 0.040101695060730025)


GROUP OF WORDS: career


MODEL: Not debiased model
('ambizione', 0.033331044763326656)

MODEL: Debiased model single
('denaro', 0.008128891512751646)

MODEL: Debiased model sum
('potere', 0.014785196445882276)

MODEL: Debiased model pca
('capo', 0.03843159824609754)


GROUP OF WORDS: rage


MODEL: Not debiased model
('crudele', 0.02310821935534474)

MODEL: Debiased model single
('aggressivo', 0.005208798125386327)

MODEL: Debiased model 



GROUP OF WORDS: adj_appearence


MODEL: Not debiased model
('brutto', 0.05616789534687999)

MODEL: Debiased model single
('magro', 0.005951637960970313)

MODEL: Debiased model sum
('magro', 0.027926203981041953)

MODEL: Debiased model pca
('piacevole', 0.050618596747517564)


GROUP OF WORDS: family


MODEL: Not debiased model
('bambino', 0.16640051901340486)

MODEL: Debiased model single
('matrimonio', 1.471489663451564e-08)

MODEL: Debiased model sum
('matrimonio', 0.03363258354365828)

MODEL: Debiased model pca
('matrimonio', 0.02690003253519546)


GROUP OF WORDS: career


MODEL: Not debiased model
('capo', 0.0022651735227555148)

MODEL: Debiased model single
('potere', 0.004747527278959662)

MODEL: Debiased model sum
('denaro', 0.027800564840435915)

MODEL: Debiased model pca
('ambizione', 0.051518983021378584)


GROUP OF WORDS: rage


MODEL: Not debiased model
('odioso', 0.034859059005975745)

MODEL: Debiased model single
('aggressivo', 0.005336591415107117)

MODEL: Debiased mode



GROUP OF WORDS: adj_appearence


MODEL: Not debiased model
('splendido', 0.05066542923450465)

MODEL: Debiased model single
('rozzo', 0.00464373100548976)

MODEL: Debiased model sum
('rozzo', 0.01789615377783771)

MODEL: Debiased model pca
('rozzo', 0.03807597104460003)


GROUP OF WORDS: family


MODEL: Not debiased model
('matrimonio', 0.07286201417446136)

MODEL: Debiased model single
('matrimonio', 4.339963199928576e-08)

MODEL: Debiased model sum
('matrimonio', 0.010125575959682442)

MODEL: Debiased model pca
('matrimonio', 0.012361686490476198)


GROUP OF WORDS: career


MODEL: Not debiased model
('capo', 0.1179308181628585)

MODEL: Debiased model single
('onorevole', 0.001222359947860241)

MODEL: Debiased model sum
('onorevole', 0.01789615377783771)

MODEL: Debiased model pca
('lavoro', 0.040664596110582374)


GROUP OF WORDS: rage


MODEL: Not debiased model
('brutale', 0.015967693179845782)

MODEL: Debiased model single
('aggressivo', 0.008775549568235785)

MODEL: Debiased mod



GROUP OF WORDS: adj_appearence


MODEL: Not debiased model
('splendido', 0.006678805407136668)

MODEL: Debiased model single
('frivolo', 0.0013397276401518887)

MODEL: Debiased model sum
('rozzo', 0.006914766132831707)

MODEL: Debiased model pca
('rozzo', 0.007583683356642723)


GROUP OF WORDS: family


MODEL: Not debiased model
('accudire', 0.12301618522033098)

MODEL: Debiased model single
('matrimonio', 0.0074775189161300215)

MODEL: Debiased model sum
('matrimonio', 0.0016129970550537998)

MODEL: Debiased model pca
('matrimonio', 0.0022892255336045553)


GROUP OF WORDS: career


MODEL: Not debiased model
('ambizione', 0.0026756316423415805)

MODEL: Debiased model single
('potere', 0.0013397376984357612)

MODEL: Debiased model sum
('capo', 0.006914766132831707)

MODEL: Debiased model pca
('capo', 0.007583683356642723)


GROUP OF WORDS: rage


MODEL: Not debiased model
('odioso', 0.15286566242575642)

MODEL: Debiased model single
('cattivo', 0.0013397376984357612)

MODEL: Debiased 